In [ ]:
import os 
import matplotlib.pyplot as plt
import scipy.optimize
import pandas as pd

1. Downloaden data
2. Cleanen data
3. Feature engineering
4. Bepalen aantal stemmen per positie  
  1. Zipfs law
  1. Meindertsma model
  1. A / (X + N)
5. Univariate analyse
1. Model

## Read

In [ ]:
einde_stemperiode = (pd.read_excel('Data\EindeStemperiode.xlsx', engine='openpyxl')
                     .dropna(subset=['EindeStemperiode'])
                     .sort_values('EindeStemperiode')
                    )

In [ ]:
notering = pd.read_parquet(os.path.join('Data', 'notering.parquet'))
song = pd.read_parquet(os.path.join('Data', 'song.parquet'))
songartist = pd.read_parquet(os.path.join('Data', 'songartist.parquet'))
artist = pd.read_parquet(os.path.join('Data', 'artist.parquet'))

## Estimate votes per ranking

In [ ]:
m = MeindertsmaVotesEstimator()

In [ ]:
notering = notering.assign(PctVotes = lambda df: df['Rank'].apply(m.percentage_of_votes))

# Clean from here

In [ ]:
analysis = (notering.merge(song, left_on='SongID', right_index=True)
                    .merge(songartist.reset_index())
                    .merge(artist, left_on='ArtistID', right_index=True, suffixes=('Song', 'Artist'))
            )

In [ ]:
def feature_engineer(df):
    df = df.assign(NrArtists = lambda df: df['SongID'].map(songartist.groupby('SongID').size()))
    return df

In [ ]:
analysis = analysis.pipe(feature_engineer)

In [ ]:
analysis_solo_songs = analysis.query('NrArtists == 1').copy()

In [ ]:
overlijdens_datum_per_song = analysis.groupby('SongID')['JaarTop2000'].first()

In [ ]:
def normalize_song_by_year(series, years_to_normalize=2):
    series.index -= overlijdens_datum_per_song.loc[series.name]
    series /= series.loc[-years_to_normalize:-1].mean()
    return series


In [ ]:
songs_with_overleden_artist = overlijdens_datum_per_song[overlijdens_datum_per_song.notnull()].index

In [ ]:
song_boost = (analysis.drop_duplicates(['Year', 'SongID']).set_index(['Year', 'SongID'])['PctVotes'].unstack(fill_value=0.5*pct_votes(2000)).stack().unstack()
 [songs_with_overleden_artist].apply(normalize_song_by_year)
)

In [ ]:
song_boost.agg(['mean', 'sem', 'count'], axis='columns').plot(y='mean', yerr='sem')

In [ ]:
songartist.groupby('SongID').size()

In [ ]:
def find_before_death_song_popularity(songID):
    try:
        return notering.loc[(notering['SongID'] == songID) & (notering['Year'] == overlijdens_datum_per_song.loc[songID] - 1), 'PctVotes'].values[0]
    except IndexError:
        return 0

In [ ]:
song

In [ ]:
song_effect_overlijden = (pd.concat([song_boost.loc[0].rename('OverlijdensBoost'),
                                     overlijdens_datum_per_song,
                                     songartist.groupby('SongID').size().rename('AantalArtiesten'),
                                    song['YearMade']], axis='columns', join='inner')
                             .assign(BeforeDeathPopularity = lambda df: [find_before_death_song_popularity(songid) for songid in df.index],
                                    )
                            )

In [ ]:
song_effect_overlijden

In [ ]:
song_effect_overlijden.groupby(song_effect_overlijden['YearMade'] // 10 * 10)['OverlijdensBoost'].agg(['mean', 'sem', 'std', 'count']).plot(y='mean', yerr='sem')

In [ ]:
song_effect_overlijden.groupby('AantalArtiesten')['OverlijdensBoost'].agg(['mean', 'sem', 'std', 'count'])

In [ ]:
song_effect_overlijden.groupby('JaarTop2000')['OverlijdensBoost'].agg(['mean', 'sem', 'std', 'count'])

In [ ]:
song_effect_overlijden.plot(x='BeforeDeathPopularity', y='OverlijdensBoost', kind='scatter')

In [ ]:
song_effect_overlijden.groupby('AantalArtiesten')['OverlijdensBoost'].agg(['mean', 'sem', 'std', 'count'])

In [ ]:
effect_overlijden = (pd.concat([song_boost.loc[0].rename('OverlijdensBoost'),
                                artist.loc[artist['IsOverleden'] == True, ['Name', 'Overleden', 'IsNederlands', 'EindeStemperiode', 'JaarTop2000']]],
                               axis='columns')
                     .assign(DaysToStemperiode = lambda df: df['Overleden'].sub(df['EindeStemperiode']).dt.days,
                            BeforeDeathPopularity = lambda df: [find_popularity_before_passing_away(artistname) for artistname in df.index],
                            LogPopularity = lambda df: np.log10(df['BeforeDeathPopularity']))
                     .replace([np.inf, -np.inf], np.nan)
                     .dropna()
                    )

In [ ]:
votes_per_year = analysis_solo_songs.groupby(['Year', 'ArtistID'])['PctVotes'].sum().unstack(fill_value=0)

In [ ]:
votes_per_year[votes_per_year.sum().nlargest(10).index].rename(columns=artist['Name'].to_dict()).plot(figsize=(20, 6))

In [ ]:
def mape_by_window(df, window=3):
    average = np.around(votes_per_year.rolling(window).mean().shift(), 5)
    mape = (votes_per_year.sub(average).abs()#.div(average) # 
             .melt()
             .replace([np.inf, -np.inf], np.nan).dropna()
             .mean()
           )
    return mape

In [ ]:
{i: mape_by_window(votes_per_year, i) for i in range(1, 5)}

In [ ]:
artist.rename(columns = lambda s: s.lower()).filter(regex='(?<!achtergrond)(?<!koor)zang')#.columns.tolist()

In [ ]:
artist[artist['gitarist, tweede leadzang'].notnull()]

In [ ]:
window=2
average = np.around(votes_per_year.rolling(window).mean().shift(), 5)
votes_per_year.sub(average).abs().div(average).melt().replace([np.inf, -np.inf], np.nan).dropna().mean()#ignore_index=False)

In [ ]:
# outc = votes_per_year.sub(average).abs().div(average).reset_index().melt(id_vars=['Jaar']).replace([np.inf, -np.inf], np.nan).dropna()

In [ ]:
# def normalize(series, years_to_normalize=3):
#     series.index -= series.idxmax()
#     series /= series.loc[-years_to_normalize:-1].mean()
#     return series

# normalize(votes_per_year['Ramses Shaffy'])

In [ ]:
artist['IsOverleden'].sum()

In [ ]:
artist

In [ ]:
overleden_artiest = artist[artist['IsOverleden'] == True].index

In [ ]:
overleden_artiest

In [ ]:
def normalize_by_year(series, years_to_normalize=3):
    series.index -= artist.loc[series.name, 'JaarTop2000']
    series /= series.loc[-years_to_normalize:-1].mean()
    return series

overleden_artiest = votes_per_year.columns.intersection(overleden_artiest)

votes_per_year_normalized = votes_per_year[overleden_artiest].apply(normalize_by_year, years_to_normalize=2)

In [ ]:
votes_per_year_normalized.replace(np.inf, np.nan).agg(['mean', 'sem', 'std', 'count'], axis='columns')#.plot(y='mean', yerr='std')

In [ ]:
def find_popularity_before_passing_away(artistname, nr_years=2):
    year_passing_away = artist.loc[artistname, 'JaarTop2000']
    popularity = votes_per_year.loc[year_passing_away-nr_years:year_passing_away-1, artistname].mean()
    return popularity

In [ ]:
effect_overlijden = (pd.concat([votes_per_year_normalized.loc[0].rename('OverlijdensBoost'),
                                artist.loc[artist['IsOverleden'] == True, ['Name', 'Overleden', 'IsNederlands', 'EindeStemperiode', 'JaarTop2000']]],
                               axis='columns', join='inner')
                     .assign(DaysToStemperiode = lambda df: df['Overleden'].sub(df['EindeStemperiode']).dt.days,
                            BeforeDeathPopularity = lambda df: [find_popularity_before_passing_away(artistname) for artistname in df.index],
                            LogPopularity = lambda df: np.log10(df['BeforeDeathPopularity']))
                     .sort_values('BeforeDeathPopularity')
                     .drop_duplicates('Overleden') # TODO: fix the double artists in a neat way

                     .replace([np.inf, -np.inf], np.nan)
                     .dropna()
                    )

In [ ]:
effect_overlijden

In [ ]:
def death_effect(x, minimum_effect=1.5, maximum_effect=3, days_start=30):
    return minimum_effect + x.add(days_start).clip(lower=0).div(days_start) * (maximum_effect - minimum_effect)

In [ ]:
popt, pcov = scipy.optimize.curve_fit(death_effect, effect_overlijden['DaysToStemperiode'], effect_overlijden['OverlijdensBoost'], [1.5, 3, 30])
perr = np.sqrt(np.diag(pcov))
popt

In [ ]:
perr

In [ ]:
recency_days = 50
effect_overlijden = effect_overlijden.assign(RecencyFactor = lambda df: df['DaysToStemperiode'].add(recency_days).clip(lower=0).div(recency_days)).sort_values('RecencyFactor')

In [ ]:
effect_overlijden.groupby('IsNederlands')['OverlijdensBoost'].agg(['mean', 'sem', 'count'])

In [ ]:
effect_overlijden.sort_values('OverlijdensBoost')

In [ ]:
effect_overlijden.plot(x='BeforeDeathPopularity', y='OverlijdensBoost', kind='scatter', logx=True)# y.assign().plot(x='LogPopularity'

In [ ]:
effect_overlijden.plot(x='JaarTop2000', y='OverlijdensBoost', kind='scatter', ylim=(0, 5))

In [ ]:
effect_overlijden.groupby('JaarTop2000')['OverlijdensBoost'].agg(['mean', 'sem'])#.plot(y='mean', yerr='sem')

In [ ]:
import statsmodels.api as sm

In [ ]:
effect_overlijden[['RecencyFactor', 'IsNederlands', 'JaarTop2000', 'LogPopularity']].corr()

In [ ]:
effect_overlijden.sort_values('OverlijdensBoost', ascending=False)

In [ ]:
y = effect_overlijden['OverlijdensBoost']
X = sm.add_constant(effect_overlijden.assign(IsNederlands = lambda df: df['IsNederlands'].astype(int),
                                             JaarTop2000 = lambda df: df['JaarTop2000'].sub(df['JaarTop2000'].max()),
                                            )
                    [['RecencyFactor', 'IsNederlands', 'JaarTop2000', 'LogPopularity']]
#                     [['RecencyFactor', 'JaarTop2000']]

                   )
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()

In [ ]:
effect_overlijden['PredictedBoost'] = res.predict(X)

In [ ]:
(effect_overlijden
 .assign(AbsoluteError = lambda df: df['OverlijdensBoost'].sub(df['PredictedBoost']).abs())
 .sort_values('AbsoluteError', ascending=False)
 .tail(20)
)#.plot(kind='scatter', x='DaysToStemperiode', y='AbsoluteError')

In [ ]:
effect_overlijden.plot(x='PredictedBoost', y='OverlijdensBoost', kind='scatter')
import matplotlib.pyplot as plt
plt.plot([0, 10], [0, 10], 'k--')

In [ ]:
effect_overlijden.groupby(effect_overlijden['Overleden'].dt.year)['OverlijdensBoost'].agg(['mean', 'sem', 'count'])#.rolling(3)['mean'].mean().plot()

In [ ]:
import datetime
import matplotlib.pyplot as plt
(effect_overlijden
 .plot(x='DaysToStemperiode', y='OverlijdensBoost', kind='scatter')
)
death_effect(pd.Series(range(-365, 1), index=range(-365, 1)), *popt).plot()
plt.axhline(1, c='k', ls='--')